In [1]:
import json

In [16]:
output_file_2 = "/workspaces/RAG_LangChain/data/chunks_docs/chunks_golondrinas_v2.json"
from unstructured.staging.base import dict_to_elements, elements_to_json

with open(output_file_2, 'r', encoding='utf-8') as f:
    json_data = f.read()

# Convertir la cadena JSON en una lista de diccionarios
element_dicts = json.loads(json_data)

# Verificar que element_dicts es una lista de diccionarios
if isinstance(element_dicts, list) and all(isinstance(d, dict) for d in element_dicts):
    # Convertir los diccionarios en elementos utilizando dict_to_elements
    loaded_chunks = dict_to_elements(element_dicts)
else:
    raise ValueError("El contenido del archivo JSON no es una lista de diccionarios válida.")

In [18]:
len(loaded_chunks)

35

In [ ]:
# Get the images from the CompositeElement objects
def get_images_base64(chunks):
    images_b64 = []
    for chunk in chunks:
        if "CompositeElement" in chunk['type']:
            chunk_els = chunks
            for el in chunk_els:
                if "Image" in str(type(el)):
                    images_b64.append(el.metadata.image_base64)
    return images_b64

images = get_images_base64(chunks_data)

In [5]:
images

[]